In [1]:
import pandas as pd
def load_tech():
    return (
        pd.read_csv("./dataframes/tech_coeff_pri.csv"), 
        pd.read_csv("./dataframes/tech_coeff_sec.csv"),
        pd.read_csv("./dataframes/tech_coeff_ter.csv"),
        pd.read_csv("./dataframes/reuse.csv")
        )

In [2]:
tech_primary, tech_secondary, tech_tertiary, reuse_df = load_tech()

In [3]:
tech_primary.columns

Index(['Tech', 'Land', 'Power', 'Capital Cost', 'O&M Cost', 'BOD(mg/l)',
       'COD(mg/l)', 'TSS(mg/l)', 'TN(mg/l)', 'FC(MPN/l)'],
      dtype='object')

In [7]:
waste_labels = ["BOD(mg/l)", "COD(mg/l)", "TSS(mg/l)", "TN(mg/l)", "FC(MPN/l)"]
dev_labels = ['Land', 'Power', 'Capital Cost', 'O&M Cost']

In [39]:
from itertools import product
combinations = product(*(tech_primary["Tech"].to_list(), tech_secondary["Tech"].to_list(), tech_tertiary["Tech"].to_list()))
coeff_df = pd.DataFrame(columns=["Tech Stack"]+waste_labels+dev_labels)

In [40]:
for p, s, t in combinations:
    c_p = 1 - (tech_primary[tech_primary["Tech"] == p][waste_labels].to_numpy()/100)
    c_s = 1 - (tech_secondary[tech_secondary["Tech"] == s][waste_labels].to_numpy()/100)
    c_t = 1 - (tech_tertiary[tech_tertiary["Tech"] == t][waste_labels].to_numpy()/100)

    d_p = tech_primary[tech_primary["Tech"] == p][dev_labels].to_numpy()
    d_s = tech_secondary[tech_secondary["Tech"] == s][dev_labels].to_numpy()
    d_t = tech_tertiary[tech_tertiary["Tech"] == t][dev_labels].to_numpy()

    d = d_p + d_s + d_t

    coeff = c_p * c_s * c_t
    t = t.replace(" + ", "&")
    _text = f"{p}+{s}+{t}"
    data = {"Tech Stack": _text}
    for i, label in enumerate(waste_labels):
        data[label] = round(coeff[0, i], 10)

    for i, label in enumerate(dev_labels):
        data[label] = d[0, i]

    coeff_df = coeff_df.append(data, ignore_index=True)

In [41]:
data

{'Tech Stack': 'Trickling Filter+WUHERMAN+mf&ro',
 'BOD(mg/l)': 0.0034126294,
 'COD(mg/l)': 0.20613822,
 'TSS(mg/l)': 0.0001684763,
 'TN(mg/l)': 0.074925,
 'FC(MPN/l)': 1e-06,
 'Land': 0.62,
 'Power': 450.0,
 'Capital Cost': 31.18,
 'O&M Cost': 2.0700000000000003}

In [42]:
coeff_df

,Tech Stack,BOD(mg/l),COD(mg/l),TSS(mg/l),TN(mg/l),FC(MPN/l),Land,Power,Capital Cost,O&M Cost
0,ASP+None+None,0.216300,0.159100,0.122400,0.900000,0.099800,0.200,202.5,5.37,0.40
1,ASP+None+Coagulation&Flocculation,0.075705,0.074777,0.058581,0.900000,0.019960,0.300,342.5,8.95,1.30
2,ASP+None+uf&ro,0.058855,0.062097,0.018360,0.630000,0.000010,0.250,382.5,23.28,1.47
3,ASP+None+mf&ro,0.095691,0.092803,0.029400,0.675000,0.000010,0.280,352.5,21.49,1.47
4,ASP+MLE+None,0.216300,0.159100,0.122400,0.220500,0.009980,0.240,322.5,11.37,1.00
...,...,...,...,...,...,...,...,...,...,...
355,Trickling Filter+BARDENPH+mf&ro,0.126394,0.206138,0.028079,0.253125,0.000001,0.679,480.0,32.18,1.67
356,Trickling Filter+WUHERMAN+None,0.007714,0.353400,0.000701,0.099900,0.010000,0.540,300.0,15.06,1.00
357,Trickling Filter+WUHERMAN+Coagulation&Floccula...,0.002700,0.166098,0.000336,0.099900,0.002000,0.640,440.0,18.64,1.90
358,Trickling Filter+WUHERMAN+uf&ro,0.002099,0.137932,0.000105,0.069930,0.000001,0.590,480.0,32.97,2.07


In [43]:
coeff_df.to_csv("./dataframes/tech_stack_coeff.csv", index=False)

In [12]:
import numpy as np

In [44]:
pd.Series([1,2,45])

0     1
1     2
2    45
dtype: int64

In [47]:
import numpy as np

In [51]:

desired_reuse = {
  "BOD(mg/l)": 10,
  "COD(mg/l)": 30,
  "TSS(mg/l)": 10,
  "TN(mg/l)": 10,
  "FC(MPN/l)": 2000
}
raw_waste_inputs = {
  "BOD(mg/l)": 21,
  "COD(mg/l)": 45,
  "TSS(mg/l)": 24,
  "TN(mg/l)": 10,
  "FC(MPN/l)": 998000
}
desired_reuse_array = np.array([[v for k, v in desired_reuse.items()] for _ in range(len(coeff_df))])
raw_waste_array = np.array([[v for k, v in raw_waste_inputs.items()] for _ in range(len(coeff_df))])
after_treatment = coeff_df[waste_labels] * raw_waste_array

In [60]:
np.all(after_treatment.to_numpy()[i] < desired_reuse_array[i])

True

In [63]:
s = []

for i in range(len(coeff_df)):
    s.append(np.all(after_treatment.to_numpy()[i] < desired_reuse_array[i]))

pd.Series(s)

0      False
1      False
2       True
3       True
4      False
       ...  
355     True
356    False
357     True
358     True
359     True
Length: 360, dtype: bool